In [1]:
import pandas as pd
import numpy as np
from random import randint
from scipy import stats
import io

In [19]:
from google.colab import files
  
  
uploaded = files.upload()

Saving control_group_ab.csv to control_group_ab (1).csv


In [20]:
control = pd.read_csv(io.BytesIO(uploaded['control_group_ab.csv']))
control.head()

,Campaign Name,Date,Spend [USD],# of Impressions,Reach,# of Website Clicks,# of Searches
0,Control Campaign,1.08.2019,2280,82702,56930,7016,2290
1,Control Campaign,2.08.2019,1757,121040,102513,8110,2033
2,Control Campaign,3.08.2019,2343,131711,110862,6508,1737
3,Control Campaign,4.08.2019,1940,72878,61235,3065,1042
4,Control Campaign,6.08.2019,3083,109076,87998,4028,1709


In [21]:
from google.colab import files
  
  
uploaded = files.upload()

Saving test_group_ab.csv to test_group_ab (1).csv


In [22]:
experiment = pd.read_csv(io.BytesIO(uploaded['test_group_ab.csv']))
experiment.head()

,Campaign Name,Date,Spend [USD],# of Impressions,Reach,# of Website Clicks,# of Searches
0,Test Campaign,1.08.2019,3008,39550,35820,3038,1946
1,Test Campaign,2.08.2019,2542,100719,91236,4657,2359
2,Test Campaign,3.08.2019,2365,70263,45198,7885,2572
3,Test Campaign,4.08.2019,2710,78451,25937,4216,2216
4,Test Campaign,6.08.2019,2458,42684,31489,7488,1854


'# of Impressions' = Jumlah user yang melihat iklan
<br> Reach = Jumlah unique id (user) yang melihat iklan
<br> '# of Website Clicks' = Jumlah user yang mengklik iklan

### **General Hypotesis Testing**

In [23]:
def var_gen(x,y):
    varx = np.var(x,ddof=1)
    var_mean_x = varx/y
    return var_mean_x

### **Delta Method**

In [7]:
def var_ratio(x,y): #x/y
    mean_x = np.mean(x)
    mean_y = np.mean(y)
    var_x = np.var(x,ddof=1)
    var_y = np.var(y,ddof=1)
    cov_xy = np.cov(x,y,ddof=1)[0][1]
    result = (var_x/mean_x**2 + var_y/mean_y**2 - 2*cov_xy/(mean_x*mean_y))*(mean_x*mean_x)/(mean_y*mean_y*len(x))
    return result

In [8]:
def ttest(mean_control,mean_experiment,var_control,var_experiment):
    diff = mean_experiment - mean_control
    var = var_control+var_experiment
    stde = 1.96*np.sqrt(var)
    lower = diff - stde 
    upper = diff + stde
    z = diff/np.sqrt(var)
    p_val = stats.norm.sf(abs(z))*2

    result = {'mean_control':mean_control,
             'mean_experiment':mean_experiment,
             'var_control':var_control,
             'var_experiment':var_experiment,
             'difference':diff,
             'lower_bound':lower,
             'upper_bound':upper,
             'z_value':z,
             'p-value':p_val}
    return pd.DataFrame(result,index=[0])

### **Pendapatan**

Hipotesis:
<br>
<br> H0 : μ(eks) - μ(con) <= 0
<br> H1 : μ(eks) - μ(con) > 0

<br> Gunakan taraf nyata (α) = 0.5 (5%) dengan menggunakan z-test dikarenakan jumlah sampel yang digunakan berjumlah lebih dari 30

In [24]:
var_control = var_gen(control['Spend [USD]'],len(control.index))
var_experiment = var_gen(experiment['Spend [USD]'],len(experiment.index))
mean_control = np.mean(control['Spend [USD]'])
mean_experiment = np.mean(experiment['Spend [USD]'])


ttest(mean_control,mean_experiment,var_control,var_experiment)

,mean_control,mean_experiment,var_control,var_experiment,difference,lower_bound,upper_bound,z_value,p-value
0,2304.068966,2572.241379,4557.157466,4252.065653,268.172414,84.211789,452.133039,2.857231,0.004274


Metode periklanan baru yang digunakan mampu meningkatkan pendapatan dari perusahaan

### **Numbers of Clicks (Peningkatan Clickthrough rate atau CTR)**

Hipotesis:
<br>
<br> H0 : μ(eks) - μ(con) <= 0
<br> H1 : μ(eks) - μ(con) > 0

<br> Gunakan taraf nyata (α) = 0.5 (5%) dengan menggunakan z-test dikarenakan jumlah sampel yang digunakan berjumlah lebih dari 30

In [25]:
var_control = var_ratio(control['# of Website Clicks'],control['# of Impressions'])
var_experiment = var_ratio(experiment['# of Website Clicks'],experiment['# of Impressions'])
mean_control = control['# of Website Clicks'].sum()/control['# of Impressions'].sum()
mean_experiment = experiment['# of Website Clicks'].sum()/experiment['# of Impressions'].sum()


ttest(mean_control,mean_experiment,var_control,var_experiment)

,mean_control,mean_experiment,var_control,var_experiment,difference,lower_bound,upper_bound,z_value,p-value
0,0.048565,0.082471,0.000013,0.000071,0.033906,0.015871,0.051941,3.68483,0.000229


Dengan α = 0.5 (5%) maka didapatkan z(1-α) = 1.645 yang berarti z(α) < z(value).

Sehingga tolak H0 dan terima H1. Keputusan ini juga diperkuat dengan nilai p-value yang bernilai < 0.01 yang berarti adanya bukti kuat untuk menolak H0. Jadi dapat disimpulkan bahwa tampilan iklan baru pada halaman website memberikan dampak terhadap perilaku pengunjung website untuk mengklik iklan tersebut

### **Pencarian**

Hipotesis:
<br>
<br> H0 : μ(eks) - μ(con) <= 0
<br> H1 : μ(eks) - μ(con) > 0

<br> Gunakan taraf nyata (α) = 0.5 (5%) dengan menggunakan z-test dikarenakan jumlah sampel yang digunakan berjumlah lebih dari 30

In [26]:
var_control = var_gen(control['# of Searches'],len(control.index))
var_experiment = var_gen(experiment['# of Searches'],len(experiment.index))
mean_control = np.mean(control['# of Searches'])
mean_experiment = np.mean(experiment['# of Searches'])


ttest(mean_control,mean_experiment,var_control,var_experiment)

,mean_control,mean_experiment,var_control,var_experiment,difference,lower_bound,upper_bound,z_value,p-value
0,2221.310345,2429.758621,25865.889417,5272.378461,208.448276,-137.414081,554.310633,1.181275,0.237493


Metode periklanan baru belum dapat meningkatkan hasil pencarian dalam mesin pencarian. Sehingga masih diperlukan cara yang lebih menarik untuk membuat produk perusahaan banyak dicari oleh pengguna